In [1]:
import pandas as pd
from pandas.tseries.offsets import BQuarterEnd,BMonthEnd
import datetime
from datetime import date
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
#%matplotlib inline

#** GLOBALS
ASSETS = ['SPY','IEF','TLT','DBC','GLD']
WEIGHTS = [0.3,0.15,0.4,0.075,0.075]

data = yf.download("SPY DBC TLT IEF GLD",start="2010-01-01",end="2023-01-01")
df = pd.DataFrame(data['Adj Close'])

# Quartalsenden für Rebalancing hinzufügen
rebals = []
for idx in df[df.index.month == 5].index.date:
    rebals.append(BMonthEnd().rollforward(idx))

for idx in df[df.index.month == 11].index.date:
    rebals.append(BMonthEnd().rollforward(idx))

for i, rebal_date in enumerate(rebals):
    while rebal_date not in df.index:
        rebal_date += pd.offsets.BDay()
    rebals[i] = rebal_date

df['Rebalancing_Days'] = np.where(df.index.isin(np.unique(rebals)),1,0)

# Log-Renditen berechnen
for asset in ASSETS:
    df[asset+'_ret'] = np.log(df[asset]) / np.log(df[asset].shift(1))

# Then calculate the contribution to the portfolio value of each asset.
# for the first day the contribution is obviously the dot product of the assets and the weights. On the next day the contribution is however different as the asset prices moved. That way we can better model the value of the portfolio later on

[*********************100%***********************]  5 of 5 completed


In [8]:
def calculate_ctr(df, assets, weights):
    # Create empty columns to store CTR for each asset
    for asset in assets:
        df[asset+'_CTR'] = np.nan
    
    # Initialize CTR for each asset to their respective weight on first day
    first_day_index = df.index[0]
    df.loc[first_day_index, [col for col in df.columns if '_CTR' in col]] = WEIGHTS

    # On rebalancing days, set CTR equal to specified weight
    df.loc[df['Rebalancing_Days'] == 1, [col for col in df.columns if '_CTR' in col]] = weights
    # On non-rebalancing days, calculate CTR based on previous day's weight in portfolio and asset return on current day
    for i in range(1, len(df)):
        if df.loc[df.index[i], 'Rebalancing_Days'] != 1:
            for j, asset in enumerate(assets):
                prev_weight = df.loc[df.index[i-1], asset+'_CTR'] / df.loc[df.index[i-1], asset+'_ret']
                df.loc[df.index[i], asset+'_CTR'] = prev_weight * df.loc[df.index[i], asset+'_ret']
                
    # Calculate portfolio value as the sum of CTR for all assets
    df['portfolio_value'] = df[[col for col in df.columns if '_CTR' in col]].sum(axis=1)

    return df



In [9]:
df = calculate_ctr(df, ASSETS, WEIGHTS)

In [ ]:
df[90:110]

In [10]:
df

,DBC,GLD,IEF,SPY,TLT,Rebalancing_Days,SPY_ret,IEF_ret,TLT_ret,DBC_ret,GLD_ret,SPY_CTR,IEF_CTR,TLT_CTR,DBC_CTR,GLD_CTR,portfolio_value
Date,,,,,,,,,,,,,,,,,
2009-12-31 00:00:00,23.777554,107.309998,68.913330,86.979057,63.926384,0.0,NaN,NaN,NaN,NaN,NaN,0.3,0.15,0.4,0.075,0.075,1.0
2010-01-04 00:00:00,24.376337,109.800003,69.084435,88.454185,63.869507,0.0,1.003766,1.000586,0.999786,1.007849,1.004906,NaN,NaN,NaN,NaN,NaN,0.0
2010-01-05 00:00:00,24.405312,109.699997,69.387787,88.688339,64.281990,0.0,1.000590,1.001034,1.001549,1.000372,0.999806,NaN,NaN,NaN,NaN,NaN,0.0
2010-01-06 00:00:00,24.839912,111.510002,69.107750,88.750801,63.421494,0.0,1.000157,0.999046,0.996763,1.005525,1.003484,NaN,NaN,NaN,NaN,NaN,0.0
2010-01-07 00:00:00,24.530863,110.820000,69.107750,89.125443,63.528160,0.0,1.000939,1.000000,1.000405,0.996103,0.998683,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27 00:00:00,24.740000,168.669998,95.940002,381.399994,100.139999,0.0,0.999336,0.998161,0.995683,1.002792,1.001640,NaN,NaN,NaN,NaN,NaN,0.0
2022-12-28 00:00:00,24.459999,167.910004,95.709999,376.660004,99.550003,0.0,0.997896,0.999474,0.998717,0.996452,0.999119,NaN,NaN,NaN,NaN,NaN,0.0
2022-12-29 00:00:00,24.430000,168.850006,96.180000,383.440002,100.680000,0.0,1.003008,1.001074,1.002453,0.999616,1.001090,NaN,NaN,NaN,NaN,NaN,0.0


In [ ]:
df[['portfolio_val'] + [col for col in df.columns if '_ret' in col]].plot(figsize=(10,6))
plt.ylabel('Value')
plt.show()

In [ ]:
# df['portfolio'] = df[ASSETS].dot(WEIGHTS)
# df['portfolio_ret'] = np.log(df['portfolio']) / np.log(df['portfolio'].shift(1))

In [ ]:
df_ohne_rebal = df.copy()

In [ ]:
# for rebal_date in rebals:
#     rebal_date = pd.to_datetime(rebal_date)
#     if df.loc[rebal_date, 'Rebalancing_Days'] == 1:
#         df.loc[rebal_date,'portfolio'] = df.loc[rebal_date,ASSETS].dot(WEIGHTS)

# update portfolio value on rebalancing days
for rebal_date in rebals:
    rebal_date = pd.to_datetime(rebal_date)
    df.loc[rebal_date, "portfolio"] = df.loc[rebal_date, "portfolio_ret"].cumprod() * df.loc[rebal_date, "portfolio"]

In [ ]:

for rebal_date in rebals:
    if df.loc[rebal_date, 'Rebalancing_Days'] == 1:
        # Calculate current asset prices at rebalancing date
        asset_prices = df.loc[rebal_date, ASSETS]
        # Update weights based on current asset prices
        WEIGHTS = WEIGHTS * asset_prices / df.loc[rebal_date, 'portfolio']
        # Calculate new portfolio value using updated weights
        df.loc[rebal_date:, 'portfolio'] = df.loc[rebal_date:, ASSETS].dot(WEIGHTS)


In [ ]:
df[df["Rebalancing_Days"]==1].head(5)

In [ ]:
df_ohne_rebal[df_ohne_rebal["Rebalancing_Days"]==1].head(5)

In [ ]:
df[99:105]

In [ ]:
df_ohne_rebal[99:105]